Comprehensive Analysis of the Indian Premier League and Projecting the Optimal Squad for the 2025 Season

In [1]:
# Load Dataset
import pandas as pd
dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")
df = dataset
metadata = dataset
# dataset.head()

C:\Users\niran\AppData\Local\Temp\ipykernel_26920\3160474569.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")


This dataset contains information about the IPL data, contains ball by ball data from 2008 to 2024. 


In [2]:
# This is the sample data
dataset.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We are cleaning the data and trying to create different tables namely:
Player Table 
Batsman table,
Bowlers table, 
Fielding table, 
matches table

In [3]:
# Column names and Data types 
dataset.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

1. Players Table: 
table name: player_info

In [4]:
unique_players = pd.concat([df['striker'], df['non_striker'], df['bowler']]).drop_duplicates().sort_values()

player_info = pd.DataFrame(columns=['player', 'seasons'])

player_list = []
seasons_list = []

for player in unique_players:
    player_rows = df[(df['striker'] == player) | (df['non_striker'] == player) | (df['bowler'] == player)]
    seasons = ', '.join(sorted(set(player_rows['season'].astype(str))))
    player_list.append(player)
    seasons_list.append(seasons)

player_info['player'] = player_list
player_info['seasons'] = seasons_list

player_info['seasons'] = player_info['seasons'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))

In [5]:
# Count the number of seasons a player played
player_info['num_seasons'] = player_info['seasons'].apply(lambda x: len(x.split(', ')))


In [6]:
player_info

,player,seasons,num_seasons
0,A Ashish Reddy,"2012, 2013, 2015, 2016",4
1,A Badoni,"2022, 2023, 2024",3
2,A Chandila,"2012, 2013",2
3,A Chopra,"2007/08, 2009",2
4,A Choudhary,2017,1
...,...,...,...
728,Yashpal Singh,2009,1
729,Younis Khan,2007/08,1
730,Yudhvir Singh,"2023, 2024",2
731,Yuvraj Singh,"2007/08, 2009, 2009/10, 2011, 2013, 2014, 2015...",11


Runs scored by a player in each innings ball by ball is stored in : player_runs

In [7]:
# Calculate the ball by ball runs scored by each player in each innings
player_runs = df.groupby(['match_id', 'striker'])['runs_off_bat'].apply(list).reset_index()
player_runs['runs_off_bat'] = player_runs['runs_off_bat'].apply(lambda x: ' '.join(map(str, x)))
player_runs.columns = ['match_id', 'player', 'ball_by_ball_runs']

In [8]:
player_runs

,match_id,player,ball_by_ball_runs
0,335982,AA Noffke,0 0 0 1 1 4 1 0 0 0 1 1
1,335982,B Akhil,0 0
2,335982,BB McCullum,0 0 0 0 0 0 0 4 4 6 4 0 4 1 0 6 0 6 4 0 1 1 1 ...
3,335982,CL White,0 1 0 0 1 1 1 0 2 0
4,335982,DJ Hussey,0 1 1 4 1 1 0 1 0 2 1 0
...,...,...,...
16510,1426312,SP Narine,6 0
16511,1426312,SS Iyer,4 1 1
16512,1426312,Shahbaz Ahmed,2 0 0 6 0 0 0
16513,1426312,TM Head,0


Total runs scored and runs scored in each innings is stored in: player_runs_sum

In [9]:
# Sum the runs scored by each player and calculate the total runs scored

player_runs_sum = df.groupby(['match_id', 'striker'])['runs_off_bat'].apply(list).reset_index()
player_runs_sum['runs_off_bat'] = player_runs_sum['runs_off_bat'].apply(lambda x: sum(x))
player_runs_sum.columns = ['match_id', 'player', 'runs_scored_each_inning']

# Create a new DataFrame with the total runs scored by each player
total_runs_sum = player_runs_sum.groupby('player')['runs_scored_each_inning'].sum().reset_index()
total_runs_sum.columns = ['player', 'total_runs_scored']

# Merge the two DataFrames
player_runs_sum = pd.merge(player_runs_sum, total_runs_sum, on='player', how='left')

In [10]:
player_runs_sum
store = player_runs_sum.to_csv("runs_in each_innings.csv")

In [11]:
# Calculate the balls faced in each inning
player_runs_sum['balls_faced_each_inning'] = player_runs['ball_by_ball_runs'].apply(lambda x: len(str(x).split()))

In [12]:
# Calculate the dotballs in each inning
dot_balls = df[df['runs_off_bat'] == 0].groupby(['match_id', 'striker'])['ball'].count().reset_index()
dot_balls.columns = ['match_id', 'player', 'dot_balls']
dot_balls
# Merged dotball column into players runs sum table
# player_runs_sum = pd.merge(player_runs_sum, dot_balls[['player', 'dot_balls']], on='player', how='left')


,match_id,player,dot_balls
0,335982,AA Noffke,6
1,335982,B Akhil,2
2,335982,BB McCullum,23
3,335982,CL White,5
4,335982,DJ Hussey,4
...,...,...,...
15727,1426312,Rahmanullah Gurbaz,21
15728,1426312,SP Narine,1
15729,1426312,Shahbaz Ahmed,5
15730,1426312,TM Head,1


In [13]:
player_runs_sum.head()

,match_id,player,runs_scored_each_inning,total_runs_scored,balls_faced_each_inning
0,335982,AA Noffke,9,9,12
1,335982,B Akhil,0,76,2
2,335982,BB McCullum,158,2882,77
3,335982,CL White,6,971,10
4,335982,DJ Hussey,12,1322,12


Calculate the data on bowlers stats

In [14]:
bowler_df = df
bowler_df.head(
)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.2,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.3,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.4,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.5,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,bowled,Abhishek Sharma,NaN,NaN


In [15]:
bowler_df.count()

match_id                  260920
season                    260920
start_date                260920
venue                     260920
innings                   260920
ball                      260920
batting_team              260920
bowling_team              260920
striker                   260920
non_striker               260920
bowler                    260920
runs_off_bat              260920
extras                    260920
wides                       8381
noballs                     1093
byes                         673
legbyes                     4001
penalty                        2
wicket_type                12950
player_dismissed           12950
other_wicket_type              0
other_player_dismissed         0
dtype: int64

In [16]:
bowlers_view = bowler_df[['match_id', 'season', 'innings', 'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type']]

In [17]:
bowlers_view.head()

,match_id,season,innings,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type
0,1426312,2024,1,MA Starc,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1426312,2024,1,MA Starc,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1426312,2024,1,MA Starc,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1426312,2024,1,MA Starc,2,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1426312,2024,1,MA Starc,0,0,NaN,NaN,NaN,NaN,NaN,bowled


In [18]:
# Fill missing values with 0
columns_to_fill = ['wides', 'noballs', 'byes', 'legbyes', 'penalty']
bowlers_view[columns_to_fill] = bowlers_view[columns_to_fill].fillna(0)
# Select columns
# bolwers_view_filled = bolwers_view[['match_id', 'season', 'innings', 'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type']]

C:\Users\niran\AppData\Local\Temp\ipykernel_26920\2764466983.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowlers_view[columns_to_fill] = bowlers_view[columns_to_fill].fillna(0)


In [19]:
bowlers_view

,match_id,season,innings,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type
0,1426312,2024,1,MA Starc,0,0,0.0,0.0,0.0,0.0,0.0,NaN
1,1426312,2024,1,MA Starc,0,0,0.0,0.0,0.0,0.0,0.0,NaN
2,1426312,2024,1,MA Starc,0,0,0.0,0.0,0.0,0.0,0.0,NaN
3,1426312,2024,1,MA Starc,2,0,0.0,0.0,0.0,0.0,0.0,NaN
4,1426312,2024,1,MA Starc,0,0,0.0,0.0,0.0,0.0,0.0,bowled
...,...,...,...,...,...,...,...,...,...,...,...,...
260915,335982,2007/08,2,I Sharma,0,1,0.0,0.0,0.0,1.0,0.0,NaN
260916,335982,2007/08,2,I Sharma,1,0,0.0,0.0,0.0,0.0,0.0,NaN
260917,335982,2007/08,2,I Sharma,0,0,0.0,0.0,0.0,0.0,0.0,NaN
260918,335982,2007/08,2,LR Shukla,0,1,1.0,0.0,0.0,0.0,0.0,NaN


In [20]:
# Calculate dot balls
bowlers_view['extras'] = bowlers_view['wides'] + bowlers_view['noballs'] + bowlers_view['byes'] + bowlers_view['legbyes'] + bowlers_view['penalty']

dot_balls = bowlers_view[(bowlers_view['runs_off_bat'] == 0) & (bowlers_view['extras'] == 0)]
dot_balls_count = dot_balls.groupby(['match_id', 'bowler'])['runs_off_bat'].count().reset_index(name='dot_balls')


# total_extras = df.groupby('match_id')['extras'].sum().reset_index(name='total_extras')
# # Create result dictionary
# result = {}
# result['dot_balls_count'] = dot_balls_count
# result['total_extras'] = total_extras

C:\Users\niran\AppData\Local\Temp\ipykernel_26920\1145263607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bowlers_view['extras'] = bowlers_view['wides'] + bowlers_view['noballs'] + bowlers_view['byes'] + bowlers_view['legbyes'] + bowlers_view['penalty']


In [21]:
dot_balls_count.head()

,match_id,bowler,dot_balls
0,335982,AA Noffke,6
1,335982,AB Agarkar,15
2,335982,AB Dinda,11
3,335982,I Sharma,10
4,335982,JH Kallis,8


calculte the wickets taken in each match of a player

In [22]:
# Select columns
checkruns_off_bat = bowlers_view
selected_columns = checkruns_off_bat[['match_id', 'bowler', 'innings', 'runs_off_bat', 'extras', 'wicket_type']]
# Drop rows with NaN values in 'wicket_type' column
selected_columns = selected_columns.dropna(subset=['wicket_type'])
selected_columns

,match_id,bowler,innings,runs_off_bat,extras,wicket_type
4,1426312,MA Starc,1,0,0.0,bowled
11,1426312,VG Arora,1,0,0.0,caught
28,1426312,MA Starc,1,0,0.0,caught
46,1426312,Harshit Rana,1,0,0.0,caught
66,1426312,AD Russell,1,0,0.0,caught
...,...,...,...,...,...,...
260872,335982,AB Agarkar,2,0,0.0,caught
260878,335982,AB Agarkar,2,0,0.0,caught
260892,335982,SC Ganguly,2,1,0.0,run out
260905,335982,SC Ganguly,2,0,0.0,bowled


In [23]:
# Define wicket types
wicket_types = ['bowled', 'caught', 'caught and bowled', 'hit wicket', 'lbw', 'stumped']
# Filter DataFrame for specified wicket types
wickets_df = df[df['wicket_type'].str.lower().isin(wicket_types)]
# Group by bowler and match_id, count wicket_type
wickets_by_bowler_match = wickets_df.groupby(['bowler', 'match_id'])['wicket_type'].count().reset_index()
# Rename columns
wickets_by_bowler_match.columns = ['bowler', 'match_id', 'wickets_count']
# Get all unique bowler-match combinations
all_bowlers_match = df[['bowler', 'match_id']].drop_duplicates()
# Merge with wickets_by_bowler_match and fill NaN with 0
wickets_by_bowler_match = all_bowlers_match.merge(wickets_by_bowler_match, on=['bowler', 'match_id'], how='left').fillna(0)

In [24]:
wickets_by_bowler_match.head()


,bowler,match_id,wickets_count
0,MA Starc,1426312,2.0
1,VG Arora,1426312,1.0
2,Harshit Rana,1426312,2.0
3,SP Narine,1426312,1.0
4,AD Russell,1426312,3.0


In [25]:
random_record = wickets_by_bowler_match.sample(n=1)
random_record

,bowler,match_id,wickets_count
1937,MM Ali,1304105,1.0


In [26]:
zero_wickets = wickets_by_bowler_match[wickets_by_bowler_match['wickets_count'] == 0]

In [27]:
zero_wickets

,bowler,match_id,wickets_count
6,B Kumar,1426312,0.0
8,T Natarajan,1426312,0.0
10,JD Unadkat,1426312,0.0
11,AK Markram,1426312,0.0
13,R Ashwin,1426311,0.0
...,...,...,...
12965,YK Pathan,335984,0.0
12966,D Salunkhe,335984,0.0
12967,P Kumar,335982,0.0
12971,SB Joshi,335982,0.0


In [28]:
import pandas as pd
from typing import Iterator, Any

# Assuming you have a pandas DataFrame named 'df'

valid_wickets = ['bowled', 'caught', 'caught and bowled', 'hit wicket', 'lbw', 'stumped']
df['valid_wicket'] = df['wicket_type'].apply(lambda x: x in valid_wickets)
bowler_wickets = df.groupby(['innings', 'bowler'])['valid_wicket'].sum().reset_index()
bowler_wickets = bowler_wickets[bowler_wickets['innings'].isin([1, 2])]
bowler_wickets_pivot = bowler_wickets.pivot(index='bowler', columns='innings', values='valid_wicket').fillna(0)
bowler_wickets_pivot.columns = [f'wickets_{inning}' for inning in bowler_wickets_pivot.columns]
total_wickets = df.groupby('bowler')['valid_wicket'].sum().reset_index(name='total_wickets')
bowler_stats = pd.merge(bowler_wickets_pivot, total_wickets, on='bowler')

striker_innings_stats = df.groupby(['innings', 'striker']).agg(runs=pd.NamedAgg(column='runs_off_bat', aggfunc='sum'), balls_faced=pd.NamedAgg(column='ball', aggfunc='count')).reset_index()
striker_innings_stats = striker_innings_stats[striker_innings_stats['innings'].isin([1, 2])]
striker_innings_stats['strike_rate'] = striker_innings_stats['runs'] / striker_innings_stats['balls_faced'] * 100
striker_innings_stats_pivot = striker_innings_stats.pivot(index='striker', columns='innings', values=['runs', 'balls_faced', 'strike_rate']).fillna(0)
striker_innings_stats_pivot.columns = [f'{agg}_{inning}' for (agg, inning) in striker_innings_stats_pivot.columns]

total_stats = df.groupby('striker').agg(total_runs=pd.NamedAgg(column='runs_off_bat', aggfunc='sum'), total_balls=pd.NamedAgg(column='ball', aggfunc='count')).reset_index()
total_stats['total_strike_rate'] = total_stats['total_runs'] / total_stats['total_balls'] * 100
striker_stats = pd.merge(striker_innings_stats_pivot, total_stats, on='striker')

merged_stats = pd.merge(striker_stats, bowler_stats, left_on='striker', right_on='bowler', how='outer').fillna(0)

In [29]:
bowler_balls = df.groupby(['innings', 'bowler'])['ball'].count().reset_index()
bowler_balls = bowler_balls[bowler_balls['innings'].isin([1, 2])]
bowler_balls_pivot = bowler_balls.pivot(index='bowler', columns='innings', values='ball').fillna(0)
bowler_balls_pivot.columns = [f'balls_bowled_{inning}' for inning in bowler_balls_pivot.columns]
total_balls_bowled = df['bowler'].value_counts().reset_index(name='total_balls_bowled')



In [30]:
total_balls_bowled.head()

,bowler,total_balls_bowled
0,R Ashwin,4679
1,SP Narine,4146
2,B Kumar,4060
3,RA Jadeja,3895
4,PP Chawla,3895


In [31]:
# bowler_stats = pd.merge(bowler_stats, bowler_balls_pivot, on='bowler', how='outer').fillna(0)
# bowler_stats = pd.merge(bowler_stats, total_balls_bowled, left_on='bowler', right_on='index', how='outer').fillna(0)

In [32]:
merged_stats.head()

,striker,runs_1,runs_2,balls_faced_1,balls_faced_2,strike_rate_1,strike_rate_2,total_runs,total_balls,total_strike_rate,bowler,wickets_1,wickets_2,total_wickets
0,A Ashish Reddy,166.0,114.0,115.0,81.0,144.347826,140.740741,280.0,196.0,142.857143,A Ashish Reddy,7.0,11.0,18.0
1,A Badoni,464.0,170.0,338.0,167.0,137.278107,101.796407,634.0,505.0,125.544554,A Badoni,1.0,1.0,2.0
2,A Chandila,4.0,0.0,7.0,0.0,57.142857,0.000000,4.0,7.0,57.142857,A Chandila,3.0,8.0,11.0
3,A Chopra,51.0,2.0,72.0,3.0,70.833333,66.666667,53.0,75.0,70.666667,0,0.0,0.0,0.0
4,A Choudhary,15.0,10.0,12.0,8.0,125.000000,125.000000,25.0,20.0,125.000000,A Choudhary,3.0,2.0,5.0


In [33]:
# merged_stats.to_csv("bothinnings_data.csv")

In [34]:
player_runs.head()

,match_id,player,ball_by_ball_runs
0,335982,AA Noffke,0 0 0 1 1 4 1 0 0 0 1 1
1,335982,B Akhil,0 0
2,335982,BB McCullum,0 0 0 0 0 0 0 4 4 6 4 0 4 1 0 6 0 6 4 0 1 1 1 ...
3,335982,CL White,0 1 0 0 1 1 1 0 2 0
4,335982,DJ Hussey,0 1 1 4 1 1 0 1 0 2 1 0


In [35]:
# player_runs_sum.to_csv("runs_each_innings_and_balls_faced.csv")